In [3]:
from matplotlib import pyplot
from sklearn.model_selection import train_test_split #split l'array into random train and test subsets

import tensorflow as tf
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Conv2D,Dense, Flatten, MaxPooling2D,Dropout
#from keras.Optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize
from sklearn.utils import shuffle
import random 



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [6]:
X_valid, X_train, y_valid, y_train = train_test_split(X_train,y_train, train_size=1/12)
# reshape dataset to have a single channel
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)
X_valid =X_valid.reshape(X_valid.shape[0],28,28,1)
# one hot encode target values
y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)
y_valid = np_utils.to_categorical(y_valid,10)

In [7]:
# summarize loaded dataset
print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (X_test.shape, y_test.shape))
print('val : X=%s, y=%s' % (X_valid.shape, y_valid.shape))

Train: X=(55000, 28, 28, 1), y=(55000, 10)
Test: X=(10000, 28, 28, 1), y=(10000, 10)
val : X=(5000, 28, 28, 1), y=(5000, 10)


In [8]:
# convert from integers to floats
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_valid = X_valid.astype('float32')

# normalize to range 0-1
X_train= X_train / 255.0
X_test = X_test / 255.0
X_valid= X_valid / 255.0

In [9]:
# define cnn model LeNet
def define_model():
  model = Sequential()
  model.add(Conv2D(64, (5, 5), activation='relu', input_shape=X_train.shape[1:]))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, (3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(10, activation='softmax'))
	# compile model
  # ADAM 0.001 / 0.0001
  #opt = SGD(learning_rate=0.01, momentum=0.9)
  opt=tf.keras.optimizers.Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
  model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
  return model

In [10]:
def train_model(X_train_new,Y_trian_new,filepath):
  model=define_model()
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath, save_weights_only=True, monitor='val_accuracy',mode='max',save_best_only=True)
  history_aug = model.fit(X_train_new, Y_trian_new, batch_size=64,validation_data=(X_valid,y_valid),epochs=50,callbacks=[model_checkpoint_callback], verbose=2)
  return model

In [11]:
def transpose_funct():
  name="Transpose"
  X_train_transpose=np.transpose(X_train,(0,2,1,3))
  X_train_new = np.concatenate([X_train, X_train_transpose],axis=0)
  Y_trian_new = np.concatenate([y_train, y_train], axis=0)
  X_train_trans_new, Y_trian_trans_new = shuffle(X_train_new, Y_trian_new)
  return X_train_transpose, X_train_trans_new, Y_trian_trans_new, name

In [12]:
X_train_transpose,X_train_trans_new, Y_trian_trans_new, name=transpose_funct()

In [15]:
def evaluate_model(model,name):
  scores = model.evaluate(X_test, y_test, verbose=0)
  print("Model ",name," score : %.2f%%" % (scores[1]*100))
  print("Model ",name," error rate : %.2f%%" % (100-scores[1]*100))

In [13]:
filepath='model_FMNIST_AugData_trans_CNN.h5'
model=train_model(X_train_trans_new,Y_trian_trans_new,filepath)

Epoch 1/50
1719/1719 - 37s - loss: 0.9671 - accuracy: 0.6470 - val_loss: 0.6876 - val_accuracy: 0.7378
Epoch 2/50
1719/1719 - 8s - loss: 0.6416 - accuracy: 0.7622 - val_loss: 0.5896 - val_accuracy: 0.7890
Epoch 3/50
1719/1719 - 8s - loss: 0.5667 - accuracy: 0.7941 - val_loss: 0.5249 - val_accuracy: 0.8170
Epoch 4/50
1719/1719 - 8s - loss: 0.5185 - accuracy: 0.8139 - val_loss: 0.4791 - val_accuracy: 0.8308
Epoch 5/50
1719/1719 - 8s - loss: 0.4813 - accuracy: 0.8287 - val_loss: 0.4561 - val_accuracy: 0.8382
Epoch 6/50
1719/1719 - 8s - loss: 0.4526 - accuracy: 0.8389 - val_loss: 0.4287 - val_accuracy: 0.8526
Epoch 7/50
1719/1719 - 8s - loss: 0.4319 - accuracy: 0.8454 - val_loss: 0.4078 - val_accuracy: 0.8558
Epoch 8/50
1719/1719 - 8s - loss: 0.4133 - accuracy: 0.8527 - val_loss: 0.4039 - val_accuracy: 0.8602
Epoch 9/50
1719/1719 - 8s - loss: 0.3988 - accuracy: 0.8573 - val_loss: 0.3811 - val_accuracy: 0.8690
Epoch 10/50
1719/1719 - 8s - loss: 0.3840 - accuracy: 0.8623 - val_loss: 0.3729 -

In [16]:
evaluate_model(model,name)

Model  Transpose  score : 90.58%
Model  Transpose  error rate : 9.42%


In [22]:
def flip_h():
  name="Flip_H"
  X_train_flip_h = np.flip(X_train,1)
  X_train_new = np.concatenate([X_train, X_train_flip_h],axis=0)
  Y_trian_new = np.concatenate([y_train,y_train], axis=0)
  #Mélangez les tableaux ou les matrices éparses de manière cohérente.
  X_train_new, Y_trian_new = shuffle(X_train_new, Y_trian_new)
  return X_train_flip_h, X_train_new, Y_trian_new, name

In [23]:
X_train_flip_h, X_flipH_train_new, Y_flipH_trian_new,name = flip_h()

In [24]:
def flip_v():
  name="Flip_V"
  X_train_flip_v = np.flip(X_train,2)
  X_train_new = np.concatenate([X_train, X_train_flip_v],axis=0)
  Y_trian_new = np.concatenate([y_train, y_train], axis=0)
  X_train_new, Y_trian_new = shuffle(X_train_new, Y_trian_new)
  return X_train_flip_v, X_train_new, Y_trian_new,name

In [25]:
X_train_flip_v, X_flipV_train_new, Y_flipV_trian_new , name=flip_v()

In [26]:
def gaussian_Noise(mu,sigma):
  name="Gaussian_Noise"
  gauss = np.random.normal(mu, sigma, size=X_train.shape)
  X_train_noisy = X_train * gauss
  X_train_new = np.concatenate([X_train, X_train_noisy],axis=0)
  Y_trian_new = np.concatenate([y_train, y_train], axis=0)
  X_train_new, Y_trian_new = shuffle(X_train_new, Y_trian_new)
  return  X_train_noisy,X_train_new, Y_trian_new, name

In [27]:
X_train_noisy, X_train_new, Y_trian_new, name=gaussian_Noise(1,0.05) 

In [28]:
tab=[flip_v(), flip_h(), transpose_funct(), gaussian_Noise(1,0.05)]
functionDataAug = random.choice(tab) #la methode
X_train_methode, X_Ran_train_new, Y_Ran_train_new, name=functionDataAug
name

'Flip_H'

In [ ]:
filepath='RandomChoise.h5'
model=train_model(X_Ran_train_new, Y_Ran_train_new, filepath)

In [20]:
evaluate_model(model,name)

Model  Transpose  score : 90.35%
Model  Transpose  error rate : 9.65%
